In [1]:
import time

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras

from qiskit import BasicAer
from qiskit.circuit.library import ZZFeatureMap
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua import QuantumInstance
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.datasets import ad_hoc_data
from qiskit_machine_learning.algorithms import QSVC

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn import svm

Loading Datasets

In [2]:
# Direct Kernel Set Training Data
dks_train_1 = pd.read_csv('data/Direct_Kernel_Set_I_Training.csv')

# Direct Kernel Set Classifcation Data
dks_class_1 = pd.read_csv('data/Direct_Kernel_Set_I_Classifications.csv')

x1_train = dks_class_1.iloc[:, 1:3]
y1_train = dks_class_1.iloc[:, 3]

x1_test = dks_train_1.iloc[:, 1:3]
y1_test = dks_train_1.iloc[:, 3]

x1_max = np.max(x1_train)
x1_train = x1_train/x1_max
y1_train = (y1_train + 1)/2
x1_test = x1_test/x1_max
y1_test = (y1_test + 1)/2

In [3]:
x2_train, y2_train, x2_test, y2_test = ad_hoc_data(training_size = 80, test_size = 20, n=2, gap = 0.2, one_hot = False)

In [4]:
y2_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [5]:
(x3_train, y3_train), (x3_test, y3_test) = keras.datasets.mnist.load_data()

def limit_data(x, y, nums):
    idx = (y != y)
    for i in nums:
        idx = idx | (y == i)
    return x[idx], y[idx]

# Scale images to the [0, 1] range
x3_train = x3_train.astype("float32") / 255
x3_test = x3_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x3_train = np.expand_dims(x3_train, -1)
x3_test = np.expand_dims(x3_test, -1)

y3_train = keras.utils.to_categorical(y3_train, 10)
y3_test = keras.utils.to_categorical(y3_test, 10)

Training, Timing, and Evaluating MLPs

In [6]:
mlp1 = tf.keras.Sequential([
    tf.keras.layers.Dense(8, input_shape=(2,), activation = 'tanh'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(8, activation = 'tanh'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation = "sigmoid", kernel_regularizer='l2')
])
mlp1.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.BinaryCrossentropy(), metrics=["accuracy"])
mlp1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 24        
_________________________________________________________________
batch_normalization (BatchNo (None, 8)                 32        
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
batch_normalization_1 (Batch (None, 8)                 32        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 169
Trainable params: 137
Non-trainable params: 32
_________________________________________________________________


In [7]:
batch_size = 32
epochs = 1000
start_time = time.time()
mlp1.fit(x1_train, y1_train, batch_size=batch_size, epochs=epochs) # train
train_time = time.time() - start_time
mlp1_score = mlp1.evaluate(x1_test, y1_test) #predict
test_time = time.time() - start_time - train_time

Epoch 1/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.8452 - accuracy: 0.5000
Epoch 2/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6923 - accuracy: 0.5409
Epoch 3/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6882 - accuracy: 0.5773
Epoch 4/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6797 - accuracy: 0.5864
Epoch 5/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6829 - accuracy: 0.5795
Epoch 6/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6813 - accuracy: 0.5909
Epoch 7/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6797 - accuracy: 0.5818
Epoch 8/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6759 - accuracy: 0.5864
Epoch 9/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6789 - accuracy: 0.5705
Epoch 10/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6824 - accuracy: 0.5727

14/14 [==============================] - 0s 2ms/step - loss: 0.6220 - accuracy: 0.6750
Epoch 83/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6220 - accuracy: 0.6705
Epoch 84/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6215 - accuracy: 0.6545
Epoch 85/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6416 - accuracy: 0.6386
Epoch 86/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6141 - accuracy: 0.6750
Epoch 87/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6114 - accuracy: 0.6864
Epoch 88/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6175 - accuracy: 0.6750
Epoch 89/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6219 - accuracy: 0.6636
Epoch 90/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6298 - accuracy: 0.6773
Epoch 91/1000
14/14 [==============================] - 0s 3ms/step - loss: 0.6491 - accuracy: 0.6341
Epoc

14/14 [==============================] - 0s 2ms/step - loss: 0.6101 - accuracy: 0.6773
Epoch 163/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6321 - accuracy: 0.6523
Epoch 164/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6248 - accuracy: 0.6705
Epoch 165/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6036 - accuracy: 0.6750
Epoch 166/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5895 - accuracy: 0.6909
Epoch 167/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5664 - accuracy: 0.7364
Epoch 168/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6201 - accuracy: 0.6341
Epoch 169/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6022 - accuracy: 0.6682
Epoch 170/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5913 - accuracy: 0.6864
Epoch 171/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5731 - accuracy: 0.

14/14 [==============================] - 0s 2ms/step - loss: 0.5714 - accuracy: 0.7114
Epoch 243/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5849 - accuracy: 0.6932
Epoch 244/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5989 - accuracy: 0.6818
Epoch 245/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5686 - accuracy: 0.7159
Epoch 246/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6277 - accuracy: 0.6477
Epoch 247/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5849 - accuracy: 0.6977
Epoch 248/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5917 - accuracy: 0.6818
Epoch 249/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5911 - accuracy: 0.6864
Epoch 250/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5718 - accuracy: 0.7159
Epoch 251/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5771 - accuracy: 0.

14/14 [==============================] - 0s 2ms/step - loss: 0.5851 - accuracy: 0.7045
Epoch 323/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5683 - accuracy: 0.6909
Epoch 324/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5563 - accuracy: 0.7227
Epoch 325/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.6034 - accuracy: 0.6545
Epoch 326/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5660 - accuracy: 0.7114
Epoch 327/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5719 - accuracy: 0.7045
Epoch 328/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5794 - accuracy: 0.6841
Epoch 329/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5681 - accuracy: 0.6977
Epoch 330/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5930 - accuracy: 0.6909
Epoch 331/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5844 - accuracy: 0.

...

14/14 [==============================] - 0s 2ms/step - loss: 0.5476 - accuracy: 0.7182
Epoch 643/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5418 - accuracy: 0.7114
Epoch 644/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5452 - accuracy: 0.7250
Epoch 645/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5318 - accuracy: 0.7250
Epoch 646/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5581 - accuracy: 0.7023
Epoch 647/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5369 - accuracy: 0.7136
Epoch 648/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5620 - accuracy: 0.7136
Epoch 649/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5602 - accuracy: 0.7227
Epoch 650/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5353 - accuracy: 0.7364
Epoch 651/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5482 - accuracy: 0.

14/14 [==============================] - 0s 2ms/step - loss: 0.5721 - accuracy: 0.6909
Epoch 723/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5515 - accuracy: 0.7227
Epoch 724/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5288 - accuracy: 0.7159
Epoch 725/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5678 - accuracy: 0.7182
Epoch 726/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5506 - accuracy: 0.7136
Epoch 727/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5464 - accuracy: 0.7273
Epoch 728/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5894 - accuracy: 0.6886
Epoch 729/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.4864 - accuracy: 0.7614
Epoch 730/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5474 - accuracy: 0.6955
Epoch 731/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5230 - accuracy: 0.

14/14 [==============================] - 0s 2ms/step - loss: 0.5401 - accuracy: 0.6955
Epoch 803/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5438 - accuracy: 0.6909
Epoch 804/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5675 - accuracy: 0.6932
Epoch 805/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5440 - accuracy: 0.7273
Epoch 806/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5868 - accuracy: 0.7045
Epoch 807/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5675 - accuracy: 0.6932
Epoch 808/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5355 - accuracy: 0.7295
Epoch 809/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5633 - accuracy: 0.7227
Epoch 810/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5197 - accuracy: 0.7182
Epoch 811/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5529 - accuracy: 0.

14/14 [==============================] - 0s 2ms/step - loss: 0.5631 - accuracy: 0.7136
Epoch 883/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5331 - accuracy: 0.7159
Epoch 884/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5404 - accuracy: 0.7182
Epoch 885/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5091 - accuracy: 0.7273
Epoch 886/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5860 - accuracy: 0.6886
Epoch 887/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5253 - accuracy: 0.7500
Epoch 888/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5239 - accuracy: 0.7295
Epoch 889/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5320 - accuracy: 0.7318
Epoch 890/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5410 - accuracy: 0.6955
Epoch 891/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5375 - accuracy: 0.

14/14 [==============================] - 0s 2ms/step - loss: 0.5427 - accuracy: 0.7250
Epoch 963/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5386 - accuracy: 0.7045
Epoch 964/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5443 - accuracy: 0.7045
Epoch 965/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5667 - accuracy: 0.7000
Epoch 966/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5877 - accuracy: 0.6841
Epoch 967/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5369 - accuracy: 0.7273
Epoch 968/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5342 - accuracy: 0.7318
Epoch 969/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5857 - accuracy: 0.6932
Epoch 970/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5335 - accuracy: 0.7364
Epoch 971/1000
14/14 [==============================] - 0s 2ms/step - loss: 0.5309 - accuracy: 0.

In [8]:
print ("Training time:{0}".format(train_time) + "[sec]")
print ("Test time:{0}".format(test_time) + "[sec]")

print(mlp1_score[1])

Training time:37.53255271911621[sec]
Test time:0.1670079231262207[sec]
0.699999988079071


In [9]:
mlp2 = tf.keras.Sequential([
    tf.keras.layers.Dense(8, input_shape=(2,), activation = 'tanh'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(8, activation = 'tanh'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation = "sigmoid", kernel_regularizer='l2')
])
mlp2.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.BinaryCrossentropy(), metrics=["accuracy"])
mlp2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8)                 24        
_________________________________________________________________
batch_normalization_2 (Batch (None, 8)                 32        
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 72        
_________________________________________________________________
batch_normalization_3 (Batch (None, 8)                 32        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 9         
Total params: 169
Trainable params: 137
Non-trainable params: 32
_________________________________________________________________


In [10]:
batch_size = 32
epochs = 1000
start_time = time.time()
mlp2.fit(x2_train, y2_train, batch_size=batch_size, epochs=epochs) # train
train_time = time.time() - start_time
mlp2_score = mlp2.evaluate(x2_test, y2_test) #predict
test_time = time.time() - start_time - train_time

Epoch 1/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.8350 - accuracy: 0.4187
Epoch 2/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.6907 - accuracy: 0.5437
Epoch 3/1000
5/5 [==============================] - 0s 3ms/step - loss: 0.6958 - accuracy: 0.5437
Epoch 4/1000
5/5 [==============================] - 0s 4ms/step - loss: 0.6787 - accuracy: 0.5625
Epoch 5/1000
5/5 [==============================] - 0s 3ms/step - loss: 0.6814 - accuracy: 0.5500
Epoch 6/1000
5/5 [==============================] - 0s 3ms/step - loss: 0.6810 - accuracy: 0.5375
Epoch 7/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.6632 - accuracy: 0.5875
Epoch 8/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.6673 - accuracy: 0.5875
Epoch 9/1000
5/5 [==============================] - 0s 3ms/step - loss: 0.6630 - accuracy: 0.5938
Epoch 10/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.6578 - accuracy: 0.5813
Epoch 11/1000
5/5 [

5/5 [==============================] - 0s 1ms/step - loss: 0.6359 - accuracy: 0.6687
Epoch 84/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.5761 - accuracy: 0.7312
Epoch 85/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5983 - accuracy: 0.6812
Epoch 86/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5680 - accuracy: 0.7125
Epoch 87/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5830 - accuracy: 0.6938
Epoch 88/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5693 - accuracy: 0.7125
Epoch 89/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5903 - accuracy: 0.7063
Epoch 90/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.5865 - accuracy: 0.7125
Epoch 91/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5879 - accuracy: 0.7250
Epoch 92/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5724 - accuracy: 0.7125
Epoch 93/1000
5/5 [=====

5/5 [==============================] - 0s 1ms/step - loss: 0.5458 - accuracy: 0.7312
Epoch 166/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5355 - accuracy: 0.7437
Epoch 167/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5385 - accuracy: 0.7125
Epoch 168/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5327 - accuracy: 0.7563
Epoch 169/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5501 - accuracy: 0.7437
Epoch 170/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.5264 - accuracy: 0.7688
Epoch 171/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5504 - accuracy: 0.7875
Epoch 172/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5205 - accuracy: 0.7875
Epoch 173/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5299 - accuracy: 0.7500
Epoch 174/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5191 - accuracy: 0.7500
Epoch 175/1000


5/5 [==============================] - 0s 1ms/step - loss: 0.5213 - accuracy: 0.7625
Epoch 248/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5249 - accuracy: 0.7437
Epoch 249/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5189 - accuracy: 0.7812
Epoch 250/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5402 - accuracy: 0.7563
Epoch 251/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5248 - accuracy: 0.7625
Epoch 252/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.5132 - accuracy: 0.7875
Epoch 253/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4818 - accuracy: 0.7750
Epoch 254/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5528 - accuracy: 0.7063
Epoch 255/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5424 - accuracy: 0.7375
Epoch 256/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.5148 - accuracy: 0.7688
Epoch 257/1000


5/5 [==============================] - 0s 2ms/step - loss: 0.4734 - accuracy: 0.8000
Epoch 330/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5192 - accuracy: 0.7188
Epoch 331/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4959 - accuracy: 0.7625
Epoch 332/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4720 - accuracy: 0.7688
Epoch 333/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5025 - accuracy: 0.7312
Epoch 334/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5274 - accuracy: 0.7188
Epoch 335/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.5156 - accuracy: 0.7750
Epoch 336/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.4986 - accuracy: 0.7937
Epoch 337/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.4867 - accuracy: 0.7875
Epoch 338/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.5025 - accuracy: 0.7750
Epoch 339/1000


...

5/5 [==============================] - 0s 2ms/step - loss: 0.4282 - accuracy: 0.8125
Epoch 658/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4568 - accuracy: 0.7937
Epoch 659/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.4377 - accuracy: 0.8000
Epoch 660/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.5799 - accuracy: 0.7250
Epoch 661/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.4163 - accuracy: 0.8000
Epoch 662/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4688 - accuracy: 0.7812
Epoch 663/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.4642 - accuracy: 0.7812
Epoch 664/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4565 - accuracy: 0.8000
Epoch 665/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4301 - accuracy: 0.7812
Epoch 666/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4260 - accuracy: 0.7875
Epoch 667/1000


5/5 [==============================] - 0s 2ms/step - loss: 0.4482 - accuracy: 0.7750
Epoch 740/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4337 - accuracy: 0.8188
Epoch 741/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4289 - accuracy: 0.8000
Epoch 742/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4391 - accuracy: 0.7937
Epoch 743/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4928 - accuracy: 0.7812
Epoch 744/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.5192 - accuracy: 0.7437
Epoch 745/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4254 - accuracy: 0.7812
Epoch 746/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.4829 - accuracy: 0.7812
Epoch 747/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4328 - accuracy: 0.7750
Epoch 748/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4451 - accuracy: 0.8000
Epoch 749/1000


5/5 [==============================] - 0s 2ms/step - loss: 0.4320 - accuracy: 0.7625
Epoch 822/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.4033 - accuracy: 0.7875
Epoch 823/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.3995 - accuracy: 0.8000
Epoch 824/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4002 - accuracy: 0.8000
Epoch 825/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.4306 - accuracy: 0.7750
Epoch 826/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.3998 - accuracy: 0.8313
Epoch 827/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.4097 - accuracy: 0.8062
Epoch 828/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4829 - accuracy: 0.7563
Epoch 829/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4466 - accuracy: 0.8000
Epoch 830/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.3799 - accuracy: 0.8250
Epoch 831/1000


5/5 [==============================] - 0s 1ms/step - loss: 0.4183 - accuracy: 0.7875
Epoch 904/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4689 - accuracy: 0.7500
Epoch 905/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4239 - accuracy: 0.8125
Epoch 906/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.4266 - accuracy: 0.7750
Epoch 907/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.4904 - accuracy: 0.7437
Epoch 908/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4281 - accuracy: 0.7937
Epoch 909/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.4376 - accuracy: 0.7812
Epoch 910/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.4391 - accuracy: 0.8000
Epoch 911/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.4716 - accuracy: 0.7812
Epoch 912/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.4091 - accuracy: 0.8313
Epoch 913/1000


5/5 [==============================] - 0s 2ms/step - loss: 0.4164 - accuracy: 0.7937
Epoch 986/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.3844 - accuracy: 0.8125
Epoch 987/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4221 - accuracy: 0.7750
Epoch 988/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.3813 - accuracy: 0.7937
Epoch 989/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4163 - accuracy: 0.8125
Epoch 990/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.3860 - accuracy: 0.7937
Epoch 991/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4327 - accuracy: 0.7688
Epoch 992/1000
5/5 [==============================] - 0s 2ms/step - loss: 0.3864 - accuracy: 0.8062
Epoch 993/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4322 - accuracy: 0.7875
Epoch 994/1000
5/5 [==============================] - 0s 1ms/step - loss: 0.4290 - accuracy: 0.8000
Epoch 995/1000


In [11]:
print ("Training time:{0}".format(train_time) + "[sec]")
print ("Test time:{0}".format(test_time) + "[sec]")

print(mlp2_score[1])

Training time:11.560194730758667[sec]
Test time:0.13100290298461914[sec]
0.699999988079071


In [12]:
mlp3 = tf.keras.Sequential([
    tf.keras.Input(shape=(28,28,1)),
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation="softmax"),
])
mlp3.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
mlp3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)               

In [13]:
batch_size = 128
epochs = 20
start_time = time.time()
mlp3.fit(x3_train, y3_train, batch_size=batch_size, epochs=epochs) # train
train_time = time.time() - start_time
mlp3_score = mlp3.evaluate(x3_test, y3_test) #predict
test_time = time.time() - start_time - train_time

Epoch 1/20
469/469 [==============================] - 1s 3ms/step - loss: 0.3480 - accuracy: 0.8961
Epoch 2/20
469/469 [==============================] - 1s 3ms/step - loss: 0.1066 - accuracy: 0.9673
Epoch 3/20
469/469 [==============================] - 1s 3ms/step - loss: 0.0798 - accuracy: 0.9750
Epoch 4/20
469/469 [==============================] - 1s 3ms/step - loss: 0.0685 - accuracy: 0.9789
Epoch 5/20
469/469 [==============================] - 1s 3ms/step - loss: 0.0608 - accuracy: 0.9814
Epoch 6/20
469/469 [==============================] - 1s 3ms/step - loss: 0.0544 - accuracy: 0.9830
Epoch 7/20
469/469 [==============================] - 1s 3ms/step - loss: 0.0501 - accuracy: 0.9840
Epoch 8/20
469/469 [==============================] - 1s 3ms/step - loss: 0.0447 - accuracy: 0.9857
Epoch 9/20
469/469 [==============================] - 1s 3ms/step - loss: 0.0445 - accuracy: 0.9865
Epoch 10/20
469/469 [==============================] - 1s 3ms/step - loss: 0.0409 - accuracy: 0.9869

In [14]:
print ("Training time:{0}".format(train_time) + "[sec]")
print ("Test time:{0}".format(test_time) + "[sec]")

print(mlp3_score[1])

Training time:27.258106470108032[sec]
Test time:0.5299997329711914[sec]
0.9929999709129333


Training, Timing, and Evaluating SVMs

In [15]:
svm1 = svm.SVC(C = 5, kernel = "rbf", gamma = 5)
start_time = time.time()
svm1.fit(x1_train, y1_train) # train
train_time = time.time() - start_time
y1_pred = svm1.predict(x1_test) #predict
test_time = time.time() - start_time - train_time
svm1_score = np.average(y1_pred == y1_test)

In [16]:
print ("Training time:{0}".format(train_time) + "[sec]")
print ("Test time:{0}".format(test_time) + "[sec]")

print(svm1_score)

Training time:0.009012460708618164[sec]
Test time:0.003504037857055664[sec]
0.625


In [17]:
svm2 = svm.SVC(C = 5, kernel = "rbf", gamma = 5)
start_time = time.time()
svm2.fit(x2_train, y2_train) # train
train_time = time.time() - start_time
y2_pred = svm2.predict(x2_test) #predict
test_time = time.time() - start_time - train_time
svm2_score = np.average(y2_pred == y2_test)

In [18]:
print ("Training time:{0}".format(train_time) + "[sec]")
print ("Test time:{0}".format(test_time) + "[sec]")

print(svm2_score)

Training time:0.0020003318786621094[sec]
Test time:0.0[sec]
0.9


In [19]:
(x3_train, y3_train), (x3_test, y3_test) = keras.datasets.mnist.load_data()
x3_train = x3_train.reshape((60000, 28*28))
x3_test = x3_test.reshape((-1, 28*28))
x3_train.shape
num_dims = 10
#idx = list(range(0, 28*28, int(28*28 / num_dims) + 1))
#x3_train[:10, idx]

def limit_data(x, y, nums):
    idx = (y != y)
    for i in nums:
        idx = idx | (y == i)
    return x[idx], y[idx]

x3_train, y3_train = limit_data(x3_train, y3_train, [3, 6])
x3_test, y3_test = limit_data(x3_test, y3_test, [3, 6])
y3_train = (y3_train == 3).astype(int)
y3_test = (y3_test == 3).astype(int)

scaler = StandardScaler().fit(x3_train)
x3_train = scaler.transform(x3_train)
x3_test = scaler.transform(x3_test)

pca = PCA(n_components=num_dims).fit(x3_train)
x3_train = pca.transform(x3_train)
x3_test = pca.transform(x3_test)
print(pca.explained_variance_ratio_)

samples = np.append(x3_train, x3_test, axis=0)
minmax_scale = MinMaxScaler((-1, 1)).fit(samples)
x3_train = minmax_scale.transform(x3_train)
x3_test = minmax_scale.transform(x3_test)

[0.08939126 0.05811813 0.04367666 0.02961373 0.02748219 0.02472023
 0.01852839 0.01558137 0.0149401  0.01387366]


In [20]:
svm3 = svm.SVC(C = 5, kernel = "rbf", gamma = 5)
start_time = time.time()
svm3.fit(x3_train, y3_train) # train
train_time = time.time() - start_time
y3_pred = svm3.predict(x3_test) #predict
test_time = time.time() - start_time - train_time
svm3_score = np.average(y3_pred == y3_test)

In [21]:
print ("Training time:{0}".format(train_time) + "[sec]")
print ("Test time:{0}".format(test_time) + "[sec]")

print(svm3_score)

Training time:0.20399951934814453[sec]
Test time:0.019999980926513672[sec]
0.9979674796747967
